In [4]:
import pandas as pd
import seaborn as srn
import statistics as sts

In [13]:
#importar dados e visualizar
dataset = pd.read_csv("Churn.csv", sep=";")
dataset.head()

,X0,X1,X2,X3,X4,X4.1,X6,X7,X8,X9,X10,X11
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0


In [14]:
#primeiro problema é dar nomes as colunas
dataset.columns = ["id", "score", "Estado", "Genero", "Idade", "patrímonio", "Saldo", "produtos",
                   "temcartcredito", "Ativo", "Salario", "Saiu"]

In [15]:
#vizualizar
dataset.head()

,id,score,Estado,Genero,Idade,patrímonio,Saldo,produtos,temcartcredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0


In [16]:
#salarios remover nas e substituir pela mediana
dataset["Salario"].describe()

count    9.920000e+02
mean     3.528762e+07
std      5.305800e+08
min      9.677000e+03
25%      3.029011e+06
50%      8.703250e+06
75%      1.405213e+07
max      1.193469e+10
Name: Salario, dtype: float64

In [17]:
mediana = sts.median(dataset["Salario"])
mediana

70518.0

In [18]:
#subistituir NAN por mediana
dataset["Salario"].fillna(mediana, inplace = True)

In [19]:
#verificar se NAN não existe mais
dataset["Salario"].isnull().sum()

0

In [20]:
#Genero falta de padronização e NAs
agrupado = dataset.groupby(["Genero"]).size()
agrupado


Genero
F              2
Fem            1
Feminino     461
M              6
Masculino    521
dtype: int64

In [21]:
#Total de NAs
dataset["Genero"].isnull().sum()

8

In [31]:
#preencher NAs com masculino
dataset["Genero"].fillna("masculino", inplace = True)

In [32]:
#verificar NAs novamente
dataset["Genero"].isnull().sum()                                                                                                                                                     


0

In [33]:
#padroniza de acordo com o dominio
dataset.loc[dataset["Genero"] == "M", "Genero"] = "Masculino"

In [34]:
dataset.loc[dataset["Genero"].isin( ["Fem", "F"]), "Genero"] = "Feminino"

In [35]:

agrupado = dataset.groupby(["Genero"]).size()
agrupado

Genero
Feminino     464
Masculino    527
masculino      8
dtype: int64

In [36]:
dataset.loc[dataset["Genero"] == "Masculino", "Genero"] = "masculino"

In [37]:
#vizualiza o resultado
agrupado = dataset.groupby(["Genero"]).size()
agrupado

Genero
Feminino     464
masculino    535
dtype: int64

In [40]:
#idades fora do dominio
dataset["Idade"].describe()

count    999.000000
mean      38.902903
std       11.401912
min      -20.000000
25%       32.000000
50%       37.000000
75%       44.000000
max      140.000000
Name: Idade, dtype: float64

In [41]:
#visualizar
dataset.loc[(dataset["Idade"] < 0 ) | ( dataset["Idade"] > 120) ]

,id,score,Estado,Genero,Idade,patrímonio,Saldo,produtos,temcartcredito,Ativo,Salario,Saiu
867,869,636,RS,Feminino,-10,1,17083346,1,1,0,11051028.0,1
984,986,773,RS,masculino,-20,1,12453278,2,0,1,1172357.0,0
990,992,655,RS,masculino,140,5,93147,2,1,0,6621413.0,0


In [42]:
mediana = sts.median(dataset["Idade"])
mediana

37

In [43]:
#substituir
dataset.loc[(dataset["Idade"] < 0 ) | ( dataset["Idade"] > 120 ), "Idade" ] = mediana

In [45]:
#verificar se ainda existem as idades fora do dominio

dataset.loc[(dataset["Idade"] < 0 ) | ( dataset["Idade"] > 120)]


,id,score,Estado,Genero,Idade,patrímonio,Saldo,produtos,temcartcredito,Ativo,Salario,Saiu


In [49]:

#Dados duplicados, buscamos pelo ID

dataset[dataset.duplicated(["id"], keep = False)]

,id,score,Estado,Genero,Idade,patrímonio,Saldo,produtos,temcartcredito,Ativo,Salario,Saiu
80,81,665,RS,Feminino,34,1,9664554,2,0,0,17141366.0,0
81,81,665,RS,Feminino,34,1,9664554,2,0,0,17141366.0,0


In [54]:
#excluir pelo ID
dataset.drop_duplicates( subset = "id", keep = "first", inplace = True)   

In [56]:
#buscar duplicados
dataset[dataset.duplicated(["id"], keep = False)]

,id,score,Estado,Genero,Idade,patrímonio,Saldo,produtos,temcartcredito,Ativo,Salario,Saiu


In [57]:

# Estados fora do dominio

agrupado = dataset.groupby(["Estado"]).size()
agrupado


Estado
PR    257
RP      1
RS    477
SC    258
SP      4
TD      1
dtype: int64

In [58]:
#atribuir RS 
dataset.loc[dataset["Estado"].isin (["RP", "SP", "TD"]), "Estado"] = "RS"

In [59]:
agrupado = dataset.groupby(["Estado"]).size()
agrupado

Estado
PR    257
RS    483
SC    258
dtype: int64

In [62]:
#outliers em salario, vou considerar 2 desvio padrão
desv = sts.stdev(dataset["Salario"])
desv

528988918.4679201

In [63]:
#definir padrão como maior que 2 desvio padrão
#checar se algum atende critério
dataset.loc[dataset["Salario"] >= 2 * desv]

,id,score,Estado,Genero,Idade,patrímonio,Saldo,produtos,temcartcredito,Ativo,Salario,Saiu
7,8,376,PR,Feminino,29,4,11504674,4,1,0,1.193469e+10,1
116,118,668,PR,Feminino,37,6,1678644,1,1,0,1.156383e+10,0
170,172,484,RS,Feminino,29,4,13011439,1,1,0,1.640179e+09,0
230,232,673,RS,masculino,72,1,0,2,0,1,1.119812e+09,0


In [64]:
#atualizar salarios para mediana, calculamos
mediana = sts.median(dataset["Salario"])
mediana

8637195.5

In [65]:
#atribuir
dataset.loc[dataset["Salario"] >= 2 * desv, "Salario"] = mediana


In [67]:
#remover os outliers
dataset.loc[dataset["Salario"] >= 2 * desv]

,id,score,Estado,Genero,Idade,patrímonio,Saldo,produtos,temcartcredito,Ativo,Salario,Saiu


In [68]:
dataset.head()

,id,score,Estado,Genero,Idade,patrímonio,Saldo,produtos,temcartcredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0
